# Lab 2: Linear and nonlinear dimensionality reduction

In this lab, you'll use [NumPy](https://numpy.org/) and [PyTorch](https://pytorch.org/) to build Prinicpal Components Analysis (PCA), Autoencoder, and Gaussian Process Factor Analysis (GPFA) (optional), and to a Calcium Imaging dataset or a synthetic dataset. 

In [ ]:
using_colab = False
# install some required python packages 
!pip3 install neo elephant --quiet

In [ ]:
if using_colab:
    import sys
    from google.colab import drive
    drive.mount('/content/gdrive/')
    sys.path.append('/content/gdrive/MyDrive/CSE8803_ML_for_Neural_Behavioral_Data/')

In [ ]:
from tqdm import trange
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

from scipy.io import loadmat

from elephant.gpfa import GPFA

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import numpy as np

from utils import train_ae, generate_syn_data

device = "cuda" if torch.cuda.is_available() else "cpu"

## Load the data

Load the file `v1_spike_data.mat` into Python. You'll find a matrix `Msp` , which contains a $(6400 \times 65)$ matrix of spike responses recorded from 65 neurons in primary visual cortex (V1) over 6400 time bins. Each row is a spike count from the population in a single time bin.  Each column is the spike counts from a single neuron. 

This is *real* neural data, described in [Graf *et al*, 2011](http://www.nature.com/neuro/journal/v14/n2/full/nn.2733.html).

**Some details about this dataset:**
- stimuli were drifting sinusoidal gratings at either $0^o$ or $90^o$ (randomly interleaved)
- spike counts were binned into 40ms bins
- each stimulus lasted 2560ms (64 time bins)
- each stimulus consisted of 1280ms (32 bins) of drifting sine grating, followed by 1280ms (32 bins) of blank screen
- this dataset thus has 100 stimuli, each lasting 64 bins (the first 32 time bins of which had a drifting sinusoid), hence 6400 total time bins
- this is a small subset of the data from the original paper 

Make an image showing the population response to the first 10 stimuli (640 total time bins $\times$ 64 neurons) using `plt.imshow()` (passing the argument `aspect='auto'` will make a more decipherable figure).  Add horizontal lines with `plt.axhline()` showing the start of each stimulus. 

In [ ]:
# Read in the matrix Msp from v1_spike_data.mat
if using_colab:
    spk_data = loadmat("/content/gdrive/MyDrive/CSE8803_ML_for_Neural_Behavioral_Data/data/v1_spike_data.mat")
else:
    spk_data = loadmat('./data/v1_spike_data.mat')
M = spk_data['Msp']

plt.figure()
plt.imshow(M[:640,:].T, aspect='auto')
plt.ylabel("neuron")
plt.xlabel("time bins")

### Compare the responses of two neurons

Plot the responses of neurons #9 and #33 (contained in columns 8 and 32 of `Msp`) over the first 640 time bins. Make the x-axis have units of seconds, and give the plot x and y-axis labels of "time (s)" and "spike count" respectively.  

In [ ]:
x = np.arange(0, 640*40, 40)/1000

plt.figure(figsize=(10,5))
plt.plot(x, M[:640,8], color = 'blue')
plt.plot(x, M[:640,32], color = 'red')
plt.xlabel("times (sec)"); plt.ylabel("spike count")
plt.xlim([0,640*40/1000])

Make a scatter plot showing the mean spike rate of all 65 neurons in units of spikes/s. (Use `plt.annotate()` to label each scatter points with its neuron index so individual responses can be picked out).  

In [ ]:
# get the mean spike rate in units of spikes/second (make sure the unit is correct!)
# YOUR CODE HERE
raise NotImplementedError()

plt.figure()
for i, j in enumerate(mean_spike):
    plt.plot(i, j, 'bo')
    plt.annotate(str(i), (i,j))
plt.xlabel("Neuron #"); plt.ylabel(r"Average FR")

In [ ]:
# TEST YOUR ANSWER
assert mean_spike.shape == (65,)

## Prinicpal Components Analysis (PCA)

Start by computing the covariance of the responses. You can do this using `np.cov()`, or by subtracting off the mean response from each neuron (each column of `M`) and computing $\frac{M^TM}{n − 1}$, where $n=6400$ is the number of samples (time bins).

Compute the singular values (eigenvalues), $s_i$, of the covariance matrix with `np.linalg.svd()` or `np.linalg.eig()`. 

Plot them as a function of $i$. Do the data points live close to a subspace of dimensionality less than 65?

Make a second plot showing the "cumulative fraction of variance accounted for", showing how much of the total response variance one would capture if they kept only the first $k = 1, 2, 3, \ldots , 65$ principal components (PCs).  The $k^{th}$ value of this curve is $\frac{\sum_{i=1}^k s_i}{S_{tot}}$, where $S_{tot}$ is the sum of all the singular values.

In [ ]:
# Calculate covariance and compute eigenvalues of the covariance matrix
# cov = ...
# V, s, Vt = np.linalg.svd(...)
# Vector s includes the singular values, matrix Vt represents the eigenvalues and is the transpose of V.
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST YOUR ANSWER
assert (s>=0).all()

In [ ]:
# Plot eigenvalues
plt.figure()
plt.plot(np.arange(len(s))+1,s, 'ro')
plt.xlabel("i"); plt.ylabel(r"$\lambda_i$")

print("The data live in an approximately 2 dimenisonal subspace")

# Plot cumulative variance captured
var_cap = np.cumsum(s)/np.sum(s)

plt.figure()
plt.plot(np.arange(len(s))+1,var_cap, 'ro')
plt.axhline(0.5); plt.axhline(0.9)
plt.xlabel("k"); plt.ylabel(r"Percent of variance captured")
plt.axis([0,65,0,1])

k50 = np.argmax(var_cap > 0.5) + 1
k90 = np.argmax(var_cap > 0.9) + 1
print("The first", k50, "PCs would be needed to capture 50% of the total variance.")
print("The first", k90, "PCs would be needed to capture 90% of the total variance.")

### Plot the first two principal components

Plot the first PC (i.e., the first eigenvector of the covariance matrix).  This is the pattern (across the 65 neurons) that captures the *most* variance of the time-varying population response.  

1. What does the shape of this eigenvector tell you about the dominant pattern of the response?  

2. Specifically, what does it tell us that all the elements of this vector have the same sign? You might wish to compare this to the plot of the average spike rate across neurons you made above.

*Note:* The sign of the PCs is irrelevant, since the negative of an eigenvector is also an eigenvector, so if you see that your first PC has mostly negative values I'd recommend flipping its sign so that the curve you plot has mostly positive values, which is easier to interpret.



In [ ]:
# Plot negative of first eigenvector
plt.figure()
plt.plot(-Vt[0])

# Overlay mean spike rates, normalized to be on same y-axis scale (unnecessary)
normalized_spike = mean_spike/np.max(mean_spike) * 0.4
for i, j in enumerate(normalized_spike):
    plt.plot(i, j, 'bo')
    plt.annotate(str(i), (i,j))

#### Answer the two questions above:

<span style="color:red">Your answer:</span>

### Plot the second PC.  

1. Does it look similar or different to the first PC?

2. Do neurons 9 and 33 (the two you looked at above) have similar or different loading onto this PC (i.e., do they have weights of the same or opposite sign?)

In [ ]:
# Plot second eigenvector
plt.figure()
plt.plot(Vt[1])

# Pick out particular eigenvector loadings
print("The loading of neuron #9:", Vt[1][8])
print("The loading of neuron #33:", Vt[1][32])

#### Answer the two questions above:

<span style="color:red">Your answer:</span>


### Plot the data projections onto the first two PCs

Make a scatter plot showing the scatter plot of the data onto the first two principal components. That is, plot the dot product of each data vector with the first component versus its dot product with the second component. 

These projections can be computed very easily by the matrix multiplications $M_0 u_1$ and $M_0 u_2$, where $u_1$ and $u_2$ are the first two PCs, and $M_0$ is $M$ with the mean response of each neuron subtracted off.  

1. Describe the structure observed in this plot.  Can you speculate about what is going on? (Refer to experiment description at the top, if necessary).

In [ ]:
# Compute the data projections onto the first two PCs
### 
# YOUR CODE BELOW
no_mean = M - np.mean(M,axis=0)
proj1 = no_mean @ -Vt[0]
proj2 = no_mean @ Vt[1]
### 

# Plot projection of mean corrected data on to first 2 eigenvectors
plt.figure()
plt.plot(proj1, proj2, 'bo')
plt.axis([-10,10,-10,10])
plt.xlabel("proj1"); plt.ylabel("proj2")

#### Answer the question above:

<span style="color:red">Your answer:</span>


### Plot the time traces of the projections onto the first two PCs over time for the first 2 stimuli (128 time bins). 

You should have two lines, one for proj1 and the other for proj2, plotted against time.

1. What information do you think is conveyed by the first projection? The second projection?  


In [ ]:
# Caclulate first two eigenvector projection for first tbins time bins
tbins = 128
proj1 = M[:tbins] @ -Vt[0]
proj2 = M[:tbins] @ Vt[1]

# Plot proj1 vs. proj2 to see the time trajectory (indicated by color shading)
plt.figure()
plt.plot(proj1, proj2, 'k-', alpha=0.3, zorder=-1)
plt.scatter(proj1, proj2, c= np.arange(tbins), cmap='Blues')
plt.colorbar()
plt.xlabel("proj1"); plt.ylabel("proj2")

# Plot proj1 & proj2 against time
plt.figure()
time = np.arange(0,tbins*40,40)/1000
plt.plot(time, proj1, 'r-', label="proj1")
plt.plot(time, proj2, 'b-', label="proj2")
plt.xlabel("Time (s)"); plt.ylabel("proj activation")
plt.axvline(time[64], color="black")
plt.xlim([0,5])
plt.legend()

#### Answer the question above:

<span style="color:red">Your answer:</span>


## Autoencoder

---
Autoencoder is a neural network for dimensionality reduction. The idea is that you train a neural network with the same data as inputs and outputs, but use an intermediate hidden layer (or layers) with dimensionality smaller than the original data. This forces the data through a "bottleneck" where it is represented in a low-dimensional form. This has numerous advantages:

* projectable and invertible - the link between the high/low dimensional representation is defined by the neural net
* fast and scalable - neural networks are computationally efficient
* non-linear and unsupervised - the autoencoder learns the non-linear manifold without needing labels

---

autoencoder.png

### Data Preprocessing

First, since we are dealing with Pytorch, it's necessary to pass in our data as a tensor of floats, and also remember to standardize the data.

Hint: a tensor is a container that stores data in $p$ dimensions. A matrix is a special case of a tensor that is 2D.

In [ ]:
torch.manual_seed(0)
num_stimulus = 25

neural_data_standardized = MinMaxScaler().fit_transform(M[:num_stimulus*64,:]) # 25 stimulus
neural_data_float_tensor = torch.tensor(neural_data_standardized).float()

print(neural_data_float_tensor.shape)

We need to create a custom dataset class that allows us to load our data into a pytorch data loader. Then, apply the custom dataset class to the neural data tensor.

Then, we will break the data into a train (90% of the data) and validation (10% of the data) split.

Finally, we should make a train data loader and a validation data loader, and set the batch size to be 200.

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        instance = self.data[idx, :]
        sample = {"data": instance}
        return sample

train_stimulus = np.random.choice(np.arange(num_stimulus), size=int(0.9*num_stimulus), replace=False)
validation_stimulus = np.delete(np.arange(num_stimulus), train_stimulus)

train_data = []
for index in train_stimulus:
    train_data.append(neural_data_float_tensor[index*64:(index+1)*64,:])
train_data = np.array(train_data).reshape(-1, neural_data_float_tensor.shape[1])

validation_data = []
for index in validation_stimulus:
    validation_data.append(neural_data_float_tensor[index*64:(index+1)*64,:])
validation_data = np.array(validation_data).reshape(-1,neural_data_float_tensor.shape[1])

train_dataset = CustomDataset(torch.tensor(train_data, dtype=torch.float32, device=device))
validation_dataset = CustomDataset(torch.tensor(validation_data, dtype=torch.float32, device=device))

print(len(train_data), len(validation_data))

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=200, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset = validation_dataset, batch_size=200, shuffle=True)

### Build the Autoencoder

In [ ]:
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Implementing a linear encoder.
        # Each layer is composed of a linear layer followed by a Relu activation function.
        # The last layer is just a linear layer!
        # We take a data point from a dimension of 65, to 10, to 2.

        # self.encoder = torch.nn.Sequential(
        # torch.nn.Linear(...),
        # torch.nn.ReLu(...),
        # torch.nn.Linear(...)
        #)
        # YOUR CODE HERE
        raise NotImplementedError()

        # Implementing a linear decoder (as opposed to a convolutional decoder, etc.).
        # Decoder is composed of a linear layer followed by a Relu activation function, repeated several times.
        # We take a data point from a dimension of 2, to 10, to 65.
        
        # self.decoder = torch.nn.Sequential(
        # torch.nn.Linear(...),
        # torch.nn.ReLu(...),
        # torch.nn.Linear(...),
        # torch.nn.ReLU()
        #)
        # YOUR CODE HERE
        raise NotImplementedError()

    def forward(self, x):
        latent_variables = self.encoder(x)
        reconstruction_data = self.decoder(latent_variables)
        return reconstruction_data

In [ ]:
# TEST YOUR ANSWER
torch.manual_seed(0)
model = Autoencoder().to(device)
output = model(torch.tensor(train_data, device=device))
assert output.shape == train_data.shape

### Train the Autoencoder

In [ ]:
model = Autoencoder().to(device)
model, training_losses, validation_losses = train_ae(model, train_loader, validation_loader, device, epochs=500)

# get the whole reconstructed neural data, here train_dataset.dataset.data is the whole dataset 18561 x 115.
reconstructed_data = model(train_dataset.data.to(device))
# get the latent variables of fist 3 stimulus
latent_variables = model.encoder(neural_data_float_tensor[:3*64, :].to(device))

### Plot the Training and Validation Losses


In [ ]:
plt.figure()
plt.plot(training_losses, label="train loss")
plt.plot(validation_losses, label="validation loss")
plt.legend()

### Compare the Original Neural Data and the Reconstruction Neural Data

In [ ]:
reconstructed_neural_data = reconstructed_data.cpu().detach().numpy().T
original_data = train_dataset.data.cpu().numpy().T

fig, axs = plt.subplots(1,2,figsize=(20, 5))
im = axs[0].imshow(original_data, aspect='auto') # original
fig.colorbar(im, orientation='vertical')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Neuron')
axs[0].set_title('original neural data')
im = axs[1].imshow(reconstructed_neural_data, aspect='auto') # reconstruction
fig.colorbar(im, orientation='vertical')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Neuron')
axs[1].set_title('reconstruction neural data')

### Plot the First Two Latent Variables

In [ ]:
latent_variables_cpu = latent_variables.cpu().detach().numpy()
latent1 = latent_variables_cpu[:, 0]
latent2 = latent_variables_cpu[:, 1]

plt.figure()
plt.plot(latent1, latent2, 'k-', alpha=0.3, zorder=-1)
plt.scatter(latent1, latent2, c= np.arange(len(latent1)), cmap='Blues')
plt.colorbar()
plt.xlabel("latent1"); plt.ylabel("latent2")

## Gaussian Process Factor Analysis (GPFA) (optional!)

---

### Factor Analysis (FA)

Factor analysis (FA) attempts to summarize the collective activity of a population of $N$ neurons recorded over $T$ time points using a smaller number of $D$ latent dimensions.
Similar to how we could formulate a Gaussian process as a 'latent function' to be inferred in order to explain a set of observations, we can treat FA as a problem of learning and inference in a probabilistic generative model.

In the generative model of factor analysis, the latents $X$ are _Gaussian iid_ and the firing rates $F$ are linear functions of the latents.
Finally, the actual observations $Y$ are are given by the firing rates corrupted by Gaussian noise.
We can write this mathematically as follows:

${Y} \in \mathbb{R}^{N \times T}$ (data)\
${X} \in \mathbb{R}^{D \times T}$ (latent variables)\
${C} \in \mathbb{R}^{N \times D}$ (readout matrix)

${x}_{dt} \sim \mathcal{N}(x; \mu = 0, \sigma^2=1)$ \
${y}_{nt |{C}, {\bf X}} \sim \mathcal{N}(y_{nt}; \mu = ({CX})_{nt}, \sigma_n^2)$

The corresponding marginal likelihood has a simple closed-form solution that factorizes across time:
$\log p ({Y}) = \log \int p({Y} | {X}) p({X}) d{X} = \sum_t \log \mathcal{N}({y}_t; {\mu} = 0, {\Sigma} = {C C}^T)$

This is convenient since we recall from previously that we will need to maximize this marginal likelihood with respect to our parameters to train the model (in practice there are out-of-the-box libraries such as scikit-learn which do this efficiently and robustly).

### Gaussian Process Factor Analysis (GPFA)

Gaussian process factor analysis (GPFA) attempts to ameliorate the assumption of FA that the latent states are independent across time.
This is desirable since behavioral and (presumably) cognitive processes unfold over time so we expect their latent dynamics to also be continuous in time.
Additionally, as we saw previously, building such prior information into our models can help denoise and improve inference over our hidden states.

GPFA is thus defined by a generative model where the latents $X$ are drawn from a _Gaussian process_ which introduces correlations over time.
The firing rates $F$ are linear functions of the latents as in FA, and the observation noise is still Gaussian.
Formally, we can write our generative model as follows:

${Y} \in \mathbb{R}^{N \times T}$ (data)\
${X} \in \mathbb{R}^{D \times T}$ (latent variables)\
${C} \in \mathbb{R}^{N \times D}$ (readout matrix)

${x}_d \sim \mathcal{GP}(0, K_{\text{RBF}})$, where $K_{\text{RBF}} = \exp(-\frac{(t-t^{'})^2}{2\sigma^{2}})$ is the RBF kernel function. \
${y}_{nt |{C}, {X}} \sim \mathcal{N}(y_{nt}; \mu = ({CX})_{nt}, \sigma_n^2)$

Everything is still jointly Gaussian and the marginal likelihood for GPFA also has a closed-form expression:\
$\log p ({Y}) = \log \int p({Y} | {X}) p({X}) d{X} = \log \mathcal{N}(vec({Y}); { \mu} = 0, {\Sigma} = {\bf K}_{yy})$

However, the marginal likelihood no longer factorizes across time and ${K}_{yy} \in \mathbb{R}^{NT \times NT}$ is rank $DT$.
This leads to a computational complexity of $\mathcal{O}(D^3 T^3)$ which can be prohibitive for longer time series.

---

### GPFA from a Generative Modeling Perspective

Here we provide a brief example to GPFA from a generative modeling perspective. Following the steps in Section 2:

1. Sample the latent variables ${X}$ from a _Gaussian Process_ using RBF kernel function.

2. Project the latent variables into a high-dimensional space using ${C} \in \mathbb{R}^{N \times D}$.

3. Create the observations ${Y}$ with Gaussian noise by: ${y}_{nt |{C}, {X}} \sim \mathcal{N}(y_{nt}; \mu = ({CX})_{nt}, \sigma_n^2)$.

In [ ]:
# parameters
T = 100 # number of time points
N = 20 # number of neurons
D = 2 # latent dimensionality
C = np.random.uniform(-1, 1, (N, D)) # loading matrix
sigma_n = np.random.uniform(0.2, 0.75, N) # noise parameters
ts = np.arange(T) # time series
sigma = 10 # timescale in RBF kernel function

First, we create the RBF kernel function, resulting in a $\mathbb{R}^{T \times T}$ covariance matrix $K$.

In [ ]:
# compute the RBF kernel
# K = np.exp(...)
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST YOUR ANSWER
assert K.shape == (100, 100)

Then, we generate the latent variables $X$ using the _Gaussian Process_ with RBF kernel $K$.

Hint: if $X \sim \mathcal{N}(0, K)$, then we have $X = LZ$, where $Z \sim \mathcal{N}(0, I)$. 

$L$ can be computed by cholesky decomposition with $K=LL^{T}$

In [ ]:
# compute L and X
# L = np.linalg.cholesky(...+1e-8*np.eye(T)), add a small number to prevent numerical issue
# Z = np.random.normal(...)
# X = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST YOUR ANSWER
assert X.shape == (2, 100)

Finally, we map the latent variables into a high-dimensional space using ${C}$, and create observations ${Y}$ with Gaussian noise.

In [ ]:
# map the latent variables to create observations
# F = ... # mapping X using C
# epsilon = np.random.normal(...)
# Y = F + epsilon
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# TEST YOUR ANSWER
assert Y.shape == (20, 100)

### Plot the Latent Variables $X$ and Observations $Y$

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15,5))

axs[0].set_title('2-dim latent variables')
axs[0].set_xlabel('time [s]')
for i, y in enumerate(X):
    axs[0].plot(ts, y, label=f'dimension {i}')
axs[0].legend()

axs[1].imshow(Y, cmap='Greys', aspect='auto')
axs[1].set_title('observations')
axs[1].set_xlabel('time [s]')
axs[1].set_ylabel('neurons')

### Generate Synthetic Spike Train Dataset

A synthetic spike trains could be generated by 4 steps as follows:

1. Create a time series of a 2-dimensional harmonic oscillator as latent variables.

2. Project the latent variables into a high-dimensional space via a random projection.

3. The resulting high-dimensional time series serves as time-dependent rates for an inhomogeneous multivariate Poisson process.

4. Multiple realizations of this Poisson process, which mimic spike trains from multiple trials, serve as the generated spike train data to the GPFA.

In [ ]:
num_trials = 20
num_neurons = 50
times_oscillator, oscillator_latent_2dim, spiketrains_oscillator = generate_syn_data(num_trials, num_neurons)

### Plot the Synthetic Spike Train Dataset

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5))

axs[0].set_title('2-dim Harmonic Oscillator')
axs[0].set_xlabel('time [s]')
for i, y in enumerate(oscillator_latent_2dim):
    axs[0].plot(times_oscillator, y, label=f'dimension {i}')
axs[0].legend()

axs[1].set_title('Latent variables in 2-dim space')
axs[1].set_xlabel('Dim 1')
axs[1].set_ylabel('Dim 2')
axs[1].set_aspect(1)
axs[1].plot(oscillator_latent_2dim[0], oscillator_latent_2dim[1])

trial_to_plot = 0
axs[2].set_title(f'Spike train plot of trial {trial_to_plot}')
axs[2].set_xlabel('Time (s)')
axs[2].set_ylabel('Neuron index')
for i, spiketrain in enumerate(spiketrains_oscillator[trial_to_plot]):
    axs[2].plot(spiketrain, np.ones_like(spiketrain) * i, ls='', marker='|')

### Perform GPFA on the Synthetic Spike Train Data

First, we need to create a GPFA instance.

Hint: use the GPFA in elephant package. Set the x_dim (the dimensionality of latent variables) to be 2.

In [ ]:
# create the gpfa instance
gpfa = GPFA(x_dim=2)

Then, we need to fit the GPFA using the synthetic spike train data.

Hint: use the first half of the trials for training.

In [ ]:
gpfa.fit(spiketrains_oscillator[:num_trials//2])

# print parameters
print(gpfa.params_estimated.keys())

Next, we use the remaining half of the trials as testing data, and extract their corresponding latent variables from the GPFA model.

Hint: use the transform() method in gpfa instance.

In [ ]:
latent_variables  = gpfa.transform(spiketrains_oscillator[num_trials//2:])

### Plot the Learned and the Original Latent Variables

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

linewidth_single_trial = 0.5
color_single_trial = 'C0'
alpha_single_trial = 0.5

linewidth_trial_average = 2
color_trial_average = 'C1'

axs[0].set_title('Original latent variables')
axs[0].set_xlabel('Dim 1')
axs[0].set_ylabel('Dim 2')
axs[0].set_aspect(1)
axs[0].plot(oscillator_latent_2dim[0], oscillator_latent_2dim[1])

axs[1].set_title('Learned latent variables')
axs[1].set_xlabel('Dim 1')
axs[1].set_ylabel('Dim 2')
axs[1].set_aspect(1)
# single trial latent variable
for single_trial_latent in latent_variables:
    axs[1].plot(single_trial_latent[0], single_trial_latent[1], '-', lw=linewidth_single_trial, c=color_single_trial, alpha=alpha_single_trial)
# trial averaged latent variable
average_latent_variable = np.mean(latent_variables, axis=0)
axs[1].plot(average_latent_variable[0], average_latent_variable[1], '-', lw=linewidth_trial_average, c=color_trial_average, label='Trial averaged latent variable')
axs[1].legend()